In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, model_selection, metrics
import warnings
warnings.filterwarnings("ignore")


In [63]:
energy_data = pd.read_csv('energydata_complete.csv')

In [64]:
energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


Question 12 - R2_Score for t2 and t6

In [65]:
t2 = energy_data ['T2'].values
t2 = t2.reshape(-1, 1)
t6 = energy_data['T6'].values

from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(t2, t6)
pred_t6 = linear_model.predict(t2)

In [66]:
from sklearn.metrics import r2_score
r2_score = r2_score(t6, pred_t6)
round(r2_score, 2)

0.64

In [67]:
energy_data.drop(['lights', 'date'], axis=1 , inplace= True)

In [68]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_energy_data = pd.DataFrame(scaler.fit_transform(energy_data), columns=energy_data.columns)
features_energy_data = normalised_energy_data.drop(columns=['Appliances'])
target_energy_data = normalised_energy_data['Appliances']

In [69]:
from sklearn.model_selection import train_test_split

# 70% of the data is usedfor the training of the models and the rest is used for testing
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features_energy_data, target_energy_data, test_size=0.3, random_state=42)

Question 13 - Mean Absolute Error

In [70]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [71]:
#fit the model to the training dataset
linear_model.fit(train_x, train_y)
#obtain predictions
predicted_values = linear_model.predict(test_x)

In [72]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_y, predicted_values)
round(mae, 3) 

0.05

Question 14 - RSS

In [74]:
import numpy as np
from math import sqrt
rss = np.sum(np.square(test_y - predicted_values))
round(rss, 2)

45.35

Question 15 - RMSE

In [75]:
from sklearn.metrics import  mean_squared_error
rmse = np.sqrt(mean_squared_error(test_y, predicted_values))
round(rmse, 3)

0.088

Question 16 - Coefficent of determination

In [76]:
from sklearn.metrics import r2_score
r2_score = r2_score(test_y, predicted_values)
round(r2_score, 2)

0.15

Question 17 - Min and Max Feature on linear model

In [77]:
#comparing the effects of regularisation
def get_weights_df(linear_model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(linear_model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

In [78]:
linear_model_weights = get_weights_df(linear_model, train_x, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, train_x, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, train_x, 'Lasso_weight')

In [79]:
print (linear_model_weights)

       Features  Linear_Model_Weight
0          RH_2            -0.456698
1         T_out            -0.321860
2            T2            -0.236178
3            T9            -0.189941
4          RH_8            -0.157595
5        RH_out            -0.077671
6          RH_7            -0.044614
7          RH_9            -0.039800
8            T5            -0.015657
9            T1            -0.003281
10          rv1             0.000770
11          rv2             0.000770
12  Press_mm_hg             0.006839
13           T7             0.010319
14   Visibility             0.012307
15         RH_5             0.016006
16         RH_4             0.026386
17           T4             0.028981
18    Windspeed             0.029183
19         RH_6             0.038049
20         RH_3             0.096048
21           T8             0.101995
22    Tdewpoint             0.117758
23           T6             0.236425
24           T3             0.290627
25         RH_1             0.553547


Question 18 - Ridge regression

In [80]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(train_x, train_y)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [81]:
predicted_values = ridge_reg.predict(test_x)

In [82]:
rmse = np.sqrt(mean_squared_error(test_y, predicted_values))
round(rmse, 3)

0.088

Question 19 - Lasso weight non zero

In [83]:
print (lasso_weights_df)

       Features  Lasso_weight
0        RH_out     -0.049557
1          RH_8     -0.000110
2            T1      0.000000
3     Tdewpoint      0.000000
4    Visibility      0.000000
5   Press_mm_hg     -0.000000
6         T_out      0.000000
7          RH_9     -0.000000
8            T9     -0.000000
9            T8      0.000000
10         RH_7     -0.000000
11          rv1     -0.000000
12           T7     -0.000000
13           T6      0.000000
14         RH_5      0.000000
15           T5     -0.000000
16         RH_4      0.000000
17           T4     -0.000000
18         RH_3      0.000000
19           T3      0.000000
20         RH_2     -0.000000
21           T2      0.000000
22         RH_6     -0.000000
23          rv2     -0.000000
24    Windspeed      0.002912
25         RH_1      0.017880


Question 20 - Lasso RMSE 

In [84]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(train_x, train_y)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [85]:
predicted_values = lasso_reg.predict(test_x)

In [86]:
rmse = np.sqrt(mean_squared_error(test_y, predicted_values))
round(rmse, 3)

0.094